In [8]:
!pip install moviepy


Defaulting to user installation because normal site-packages is not writeable


In [75]:
from pipeline import extract_audio
from config import VIDEO_PATH

extract_audio(VIDEO_PATH)


MoviePy - Writing audio in C:/Users/HILCPS/Downloads/audiop8209.wav


MoviePy - Done.


In [26]:
!pip install git+https://github.com/openai/whisper.git


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/whisper.git to c:\users\hilcps\appdata\local\temp\pip-req-build-qs70vtfp
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\HILCPS\AppData\Local\Temp\pip-req-build-qs70vtfp'


In [25]:
# tsv to csv conversion

In [24]:
from pipeline import tsv_to_csv
from config import TSV_PATH, CSV_PATH

tsv_to_csv(TSV_PATH, CSV_PATH)


Saved as CSV: C:\Users\HILCPS\Downloads\transcript.csv


In [ ]:
# to remove unnecessary details and to save only required time stamps and text into json file 

In [76]:
from pipeline import transcribe_audio
from config import AUDIO_PATH

transcribe_audio(AUDIO_PATH)


Cleaned transcript saved as transcript_clean_p8209.json


In [44]:
#  What It Does:
# Loads your labeled JSON (with start, end, label)

# Prompts you on the terminal to select a label (e.g., WICKET)

# Filters all events with that label

# Extracts video segments from the full video based on timestamps

# Merges all clips for that label into one single output video

In [47]:
!pip install --upgrade moviepy


Defaulting to user installation because normal site-packages is not writeable


In [58]:
!pip install ffmpeg-python


Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [81]:
from pipeline import clip_segments
from config import VIDEO_PATH, LABELED_TRANSCRIPT, CLIPS_DIR, OUTPUT_VIDEO

clip_segments(VIDEO_PATH, LABELED_TRANSCRIPT, None, CLIPS_DIR, OUTPUT_VIDEO)


🔍 Enter label to extract (e.g., SIX):  FOUR


✂️ Found 1 segments for label: FOUR
✅ Trimmed: C:\Users\HILCPS\Downloads\clips_temp\clip_000.mp4

🎬 Merged video saved to: C:\Users\HILCPS\Downloads\extracted_prompted_FOUR_ffmpeg_p8209.mp4
